<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Convolutional_Layers_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# -U: Upgrade all packages to the newest available version
!pip install -U d2l
from d2l import tensorflow as d2l
import tensorflow as tf

Requirement already up-to-date: d2l in /usr/local/lib/python3.6/dist-packages (0.15.1)


The big part of the code is from d2l.ai book : http://d2l.ai/ ; + experiments with code and some modifications are added

In [29]:
### input X and kernel K , compute cross correlation
def corr2d(X, K):  #@save
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(
                X[i: i + h, j: j + w] * K))
    return Y


In [30]:
X = tf.constant([[1., 2., 3.],[2., 3., 4.],[1., 3., 6.]])
K = tf.constant([[1.,2.],[1.,2.]])

In [31]:
corr2d(X,K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[13., 19.],
       [15., 26.]], dtype=float32)>

In [32]:
class conv2D(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()

  def build(self, kernel_size):
    initializer = tf.random_normal_initializer()
    self.weight = tf.add_weight(name='w', shape = kernel.size, initializer = initializer)
    self.bias = seld.add_weight(name = 'b', shape = (1,), initializer = initializer)

  def call(self, inputs):
    return corr2d(inputs, self.weight)+ self.bias






In [33]:
X = tf.Variable(tf.ones((6, 8)))
X[:, 2:6].assign(tf.zeros(X[:, 2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [34]:
K = tf.constant([[1.0,-1.0]])

In [35]:
Y = corr2d(X,K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [36]:
corr2d(tf.transpose(X), K)

<tf.Variable 'Variable:0' shape=(8, 5) dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

Learning Kernel to detect edges

In [37]:
### two dim conv layer with 1 output channel, 
### dimention of input is height, weight 
conv2d = tf.keras.layers.Conv2D(1,(1,2), use_bias=False)

## num-samples (batch_size), height, weight, num channels (input for X output for Y)
X = tf.reshape(X,(1,6,8,1))
Y = tf.reshape(Y,(1,6,7,1))

Y_hat = conv2d(X)

for i in range(20):
  with tf.GradientTape(watch_accessed_variables = False) as g:
    g.watch(conv2d.weights[0])
    Y_hat = conv2d(X)
    l = (abs(Y_hat - Y))**2
## update weights
    update = tf.multiply(3e-2,g.gradient(l,conv2d.weights[0]))
    weights = conv2d.get_weights()
    weights[0]=conv2d.weights[0] - update
    conv2d.set_weights(weights)
    if ((i+1)%2 == 0):
      print(f'batch{i+1}, loss {tf.reduce_sum(l):.3f}')


batch2, loss 0.200
batch4, loss 0.077
batch6, loss 0.031
batch8, loss 0.012
batch10, loss 0.005
batch12, loss 0.002
batch14, loss 0.001
batch16, loss 0.000
batch18, loss 0.000
batch20, loss 0.000


In [38]:
conv2d.get_weights()[0]

array([[[[ 1.0007808 ]],

        [[-0.99920225]]]], dtype=float32)

In [39]:
conv2d.weights[0]

<tf.Variable 'conv2d_1/kernel:0' shape=(1, 2, 1, 1) dtype=float32, numpy=
array([[[[ 1.0007808 ]],

        [[-0.99920225]]]], dtype=float32)>

In [40]:
tf.reshape(conv2d.get_weights()[0], (1,2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.0007808 , -0.99920225]], dtype=float32)>

Triangular Tensors

In [41]:
X = tf.Variable(tf.ones((10, 10)))

In [42]:
K = tf.constant([[1.0,-1.0]])

In [43]:
a = tf.linalg.band_part(X,2,0)
a

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]], dtype=float32)>

In [44]:
b = tf.linalg.band_part(X,1,1)
b

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [45]:
c = tf.linalg.band_part(X,0,-1)
c

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [46]:
d = tf.linalg.band_part(X,-1,0)
d


<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

Edge detection select diagonal in triangular matrixes

In [47]:
corr_up = corr2d(c,K)
corr_up

<tf.Variable 'Variable:0' shape=(10, 9) dtype=float32, numpy=
array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.]], dtype=float32)>

In [48]:
corr_dn = corr2d(d,K)
corr_dn

<tf.Variable 'Variable:0' shape=(10, 9) dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>